In [137]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import xarray as xr
import os
import yaml
import itertools
import shutil
from datetime import datetime
import sys

In [138]:
# ##SAVE: NEEM INTERP
# sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GISP2-NEEM.xlsx', sheet_name=2, skiprows=1, usecols=[1,2], names=['GISP2', 'NEEM2011'])

# NEEM_interp = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/GICC21 matchpoints.xlsx', sheet_name=2, skiprows=1, usecols=[9,15], names=['NEEM_og', 'NEEM2011'])
# NEEM_interp.dropna(inplace=True)

# sv_links = sv_links[sv_links['NEEM2011'] <= NEEM_interp['NEEM2011'].max()]
# sv_links['NEEM'] = np.interp(sv_links['NEEM2011'], NEEM_interp['NEEM2011'], NEEM_interp['NEEM_og'])

# sv_links = sv_links[['GISP2', 'NEEM']]

# folder='GL_20Cols'

In [ ]:
reference = 'Svensson_Links'
comment_add = 'Svensson Bipolar Link Sheet - only inter-greenland tiepoints added'

sv_links = pd.read_excel('/Users/quinnmackay/Documents/GitHub/BICC/Data Storage/Tiepoints/Svensson Links NGRIP-NEEM-GIP-GISP-EDML-EDC-WD-DF-TALDICE.xls', sheet_name=1, skiprows=0, nrows=305, usecols=[2,3], names=['GRIP', 'GISP2'])
sv_links

folder='GL_20Cols'

,GRIP,GISP2
0,1517.287,1575.712
1,1539.050,1597.200
2,1585.050,1640.880
3,1598.900,1653.880
4,1609.470,1663.790
...,...,...
300,NaN,NaN
301,NaN,NaN
302,NaN,NaN
303,NaN,NaN


In [140]:
#load cores list from params
params = f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}/parameters.yml'
with open(params, 'r') as f:
    first_line = f.readline()
params_load = yaml.safe_load(first_line)
list_sites = params_load['list_sites']

# get all link combos
pairs = [f"{a}-{b}" for a, b in itertools.combinations(list_sites, 2)]

#get all combos possible from the svensson links
subset = list(sv_links.columns)
valid_pairs = [p for p in pairs if all(site in subset for site in p.split('-'))]

In [141]:
#now, create all the link files
sv_synchros = {}
sv_comments = {}
for pair in valid_pairs:
    site_a, site_b = pair.split('-')

    # Drop NaN rows to get only valid shared tiepoints
    sv_synchros[pair] = sv_links[[site_a, site_b]].dropna()
    sv_synchros[pair].columns = ['depth1', 'depth2']
    sv_synchros[pair]['reference'] = reference  # Load reference string
    sv_comments[pair] = [comment_add]

#now load existing synchros
os.chdir(f'/Users/quinnmackay/Documents/GitHub/BICC/Antarctic Chronology Accuracy Project/{folder}')
existing_synchros = {}

for pair in valid_pairs:

    folder_path = pair
    file_path = os.path.join(folder_path, f"{reference}_{pair}.txt")
    os.makedirs(folder_path, exist_ok=True)

    if os.path.exists(file_path):
        raise FileExistsError(f"File already exists: {file_path}")

    combined = sv_synchros[pair].copy(deep=True) # svensson links load
    new_comments = sv_comments.get(pair, [])

    with open(file_path, 'w') as f:
        # Write new comments first
        for line in new_comments:
            f.write(f"#{line}\n")
        # Write merged DataFrame below
        combined.to_csv(f, sep='\t', index=False)

    print(f"Saving new synchro to {file_path}")

Saving new synchro to GISP2-GRIP/Svensson_Links_GISP2-GRIP.txt
